## Dependencies

---

In [34]:
import yaml
import re
from datetime import datetime
import pandas as pd
from pandas import DataFrame

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [36]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [37]:
# edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")
edge = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
edge.maximize_window()

### Functions:

In [38]:
def resources(name:str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content:dict[str:str] = yaml.safe_load(f)
        return content[name]

In [39]:
def close_prompt(driver:object) -> None:
    """Close ad window"""

    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [40]:
def choose_category(category:str, driver:object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [41]:
def click_element(element:object, path:str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [42]:
def extract_description(element:object, path:str) -> str:
    """Extract ad description on the current document"""

    description:str = element.find_element(by=By.XPATH, value=path)
    return description

In [43]:
def extract_feature(element:object, path:str) -> list[str]:
    """Extract some feature from list document based on xpath used."""

    feature = []

    try:
        feature:str = element.find_elements(by=By.XPATH, value=path)
        feature = [data_point.text for data_point in feature]
        return feature
    except Exception:
        feature.append("Web wasn't able to scrape the feature")

In [44]:
def persistence(dataframe:pd.DataFrame) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [45]:
def get_len(features:list[list[str]]) -> bool:
    """
    Check if all features has the same length to create new df
    :param features: list of features extracted with scraping
    :return: a list with all lengths where first place is the len of the first element and the mostly accepted.
    """

    base_len = len(features[0])
    lengths = []
    lengths.append(base_len)
    for feature in features:
        if len(feature) == base_len:
            continue
        else:
             lengths.append(len(feature))
    return lengths

In [46]:
def check_len(lengths:list[int]) -> bool:
    """
    Check if all elements in list has the same value.
    :param lengths: list of lengths
    :return: a boolean
    """
    for length in lengths[1:]:
        if length != lengths[0]:
            return False
        return True

Variables pool.

In [47]:
descriptions:list[str] = []
prices:list[int] = []
models:list[str] = []
years:list[int] = []
kilometers:list[int] = []
engines:list[str] = []

# seconds
DELAY:int = 1

In [48]:
sedan = resources("cars_sedan")

Get web & close initial prompt.

In [49]:
edge.get(sedan["url"])

In [50]:
# close_prompt(edge)

In [51]:
elements = edge.find_elements(by=By.XPATH, value=sedan["ad"])

Extract descriptions.

In [52]:
elements[0].click()
try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(EC.presence_of_element_located((By.XPATH, sedan["description"])))
        temp = extract_description(edge, sedan["description"])
        descriptions.append(temp.text)
        click_element(edge, sedan["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, sedan["exit"])

Extract prices & models.

In [53]:
prices = extract_feature(edge, sedan["price"])
models = extract_feature(edge, sedan["model"])

Extract car year, engine & kilometers.

In [54]:
years = extract_feature(edge, sedan["year"])
engines = extract_feature(edge, sedan["engine"])
kilometers = extract_feature(edge, sedan["kilometers"])

In [55]:
lengths = [prices, models, years, engines, kilometers, descriptions]
get_len(lengths)

[40, 41, 41, 37, 39]

## Export

---

### Functions:

In [56]:
def parse_price(dataframe: pd.DataFrame, feature:str, pattern:str) -> None:
    """
    Parse financial features like prices. Works with $#,### or $#,###\n* pattern
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param pattern: feature pattern to transform
    """
    dataframe[feature] = dataframe [feature] \
        .str.extract(pattern) \
        .applymap(lambda price: int(price.replace(",","")))


In [57]:
def remove_chars(dataframe: pd.DataFrame, feature:str, wildcards:str) -> pd.Series:
    """
    Remove escaped characters from features like indented large extracts of text
    :param dataframe: object towards apply transformation
    :param feature: feature name towards apply transformation
    :param wildcards: escaped characters to remove
    :return: Serie without escaped characters
    """
    return (dataframe
    .apply(
        lambda row: row[feature]
        ,axis = 1
    )
    .apply(
        lambda extracted_f: re.sub(wildcards, r' ', extracted_f)
    )
    )


In [58]:
PRICE_RG: str = r"^\$(?P<prices>\d+,\d+)?"
KM_REG: str = r"(?P<km>^\d+)"
CHARACTER_RG:str = '(\n|\r)+'

Remove noise from `model` and `years` lists

In [59]:
years.remove("Ver más")

In [60]:
models.remove("Blue Book Encuentra24")

In [100]:
lengths = [len(engines), len(models), len(years), len(kilometers), len(prices),len(descriptions)]
lengths

[40, 40, 40, 40, 40, 40]

Create dataframe to transform data extracted

In [112]:
if check_len(lengths):

    df_temp = pd.DataFrame({
        "model": models,
        "year": years,
        "kilometers": kilometers,
        "engine": engines,
        "price": prices,
        "description": descriptions
    })

    # Parse features. String to number and raw string with characters to structured string.
    parse_price(df_temp, "price", PRICE_RG)
    df_temp["kilometers"] = df_temp["kilometers"].str.extract(KM_REG).applymap(int)
    df_temp["year"] = df_temp["year"].map(int)
    df_temp["description"] = remove_chars(df_temp, "description", CHARACTER_RG)

    persistence(df_temp)


In [113]:
df_temp

,model,year,kilometers,engine,price,description
0,Mitsubishi Mirage G4,2020,14445,Gasolina,7950,Precio negociable: si Color : AZUL 3 modos de ...
1,Hyundai Elantra,2018,77000,Gasolina,10500,Color : Gris DESCRIPCIÓN ARTÍCULOS CALIFICACIO...
2,Honda Civic,2017,88590,Gasolina,11500,Precio negociable: si Color : Negro Hyundai Ac...
3,Hyundai Accent,2019,78000,Gasolina,9800,Color : Blanco Se vende Mitsubishi G4 2018. Au...
4,Mitsubishi Mirage G4,2018,60000,Gasolina,8400,Precio negociable: si DESCRIPCIÓN CALIFICACION...
5,Kia Sephia,1999,189000,Gasolina,2500,"Vehiculo de agencia, un solo dueño, solo 143 m..."
6,Mitsubishi Lancer,2008,143000,Gasolina,5900,Garantía: Como es visto no hay garantía Financ...
7,Kia Forte,2018,113000,Gasolina,9100,Precio negociable: si Color : Rojo Puede verlo...
8,Dodge DART,2013,89000,Gasolina,6200,Financiamiento: no Precio negociable: si Color...
9,BMW 328,2009,140000,Gasolina,4249,Color : Azul Vendo Mitsubishi Lancer 2017 Full...
